### Can we identify the subject based on their physiological signals and self-reports? ###

In [1]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(module_path)
import pandas as pd
import random
import scipy.signal as ss
import sys

import tools.data_reader_apd as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)

In [3]:
# LOAD TRAIN AND TEST DATA
importlib.reload(train)
importlib.reload(preprocessing)
importlib.reload(dr)
importlib.reload(dt)

# metrics = train.Metrics.ALL
metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.IBI, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
    # train.Metrics.RESP,
    train.Metrics.MEAN_ANKLE_ACT_L, 
    train.Metrics.MEAN_ANKLE_ACT_R, 
    train.Metrics.PEAK_ANKLE_ACC_L, 
    train.Metrics.PEAK_ANKLE_ACC_R,
    train.Metrics.MEAN_WRIST_ACT_L, 
    train.Metrics.MEAN_WRIST_ACT_R, 
    train.Metrics.PEAK_WRIST_ACC_L, 
    train.Metrics.PEAK_WRIST_ACC_R
]

model_phases = [
    [
        "Baseline_Rest", 
        "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
        "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
    ],
    [
        "Baseline_Rest", 
        "BugBox_Relax", "BugBox_Anticipate", "BugBox_Break",
        "Speech_Relax", "Speech_Anticipate", "Speech_Break"
    ],
    [
        "Baseline_Rest", 
        "BugBox_Relax", "BugBox_Anticipate", 
        "Speech_Relax", "Speech_Anticipate"
    ],
    [
        "Baseline_Rest", 
        "BugBox_Relax",
        "Speech_Relax"
    ],
    ["BugBox_Break", "Speech_Break"],
    ["BugBox_Exposure", "Speech_Exposure"]
]

models = {
    "SVM": SVC(C=10, gamma=1),  # C=10, gamma=1
    "KNN": KNeighborsClassifier(n_neighbors=7),
    "DT": DecisionTreeClassifier(),
    "LogReg": LogisticRegression(max_iter=1000)
}

for phases in model_phases:
    print(f"PHASES: {phases} " + "-"*30)
    anxiety_label_type = "Anxiety"
    x, y = train.Train_APD.get_apd_data_ranking(metrics, phases, verbose=False, anxiety_label_type=anxiety_label_type)
    # drop subjects with noisy data
    x = x[x['subject'] != 84.0]
    y = y[y['subject'] != 84.0]
    # x = x[x['subject'] != 8.0]
    # y = y[y['subject'] != 8.0]

    # 0-1 scaling
    for i in range(3, len(x.columns)):
        data_col = x[x.columns[i]]
        data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
        x[x.columns[i]] = data_col

    results = {
        "SVM": [], "KNN": [],
        "DT": [],
        "LogReg": []
    }
    num_iters = 10
    for _ in range(num_iters):
        out = train.train_predict(models, x, y, by_subject=True, show_classification=False)
        for model_name in results:
            results[model_name].append(out[model_name])

    for model_name in results.keys():
        print(f"{model_name} accuracy over {num_iters} rounds: {np.mean(results[model_name])}")
    print("\n")

PHASES: ['Baseline_Rest', 'BugBox_Relax', 'BugBox_Anticipate', 'BugBox_Exposure', 'BugBox_Break', 'Speech_Relax', 'Speech_Anticipate', 'Speech_Exposure', 'Speech_Break'] ------------------------------
SVM accuracy over 10 rounds: 0.7285714285714286
KNN accuracy over 10 rounds: 0.6523809523809524
DT accuracy over 10 rounds: 0.6777777777777778
LogReg accuracy over 10 rounds: 0.6603174603174604


PHASES: ['Baseline_Rest', 'BugBox_Relax', 'BugBox_Anticipate', 'BugBox_Break', 'Speech_Relax', 'Speech_Anticipate', 'Speech_Break'] ------------------------------
SVM accuracy over 10 rounds: 0.7204081632653061
KNN accuracy over 10 rounds: 0.6510204081632655
DT accuracy over 10 rounds: 0.7428571428571428
LogReg accuracy over 10 rounds: 0.6795918367346938


PHASES: ['Baseline_Rest', 'BugBox_Relax', 'BugBox_Anticipate', 'Speech_Relax', 'Speech_Anticipate'] ------------------------------
SVM accuracy over 10 rounds: 0.7171428571428571
KNN accuracy over 10 rounds: 0.6971428571428572
DT accuracy over 